In [1]:
'''
Mount Google Drive for COLAB
'''

COLAB = True

if COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)
  %cd gdrive/My\ Drive/Colab\ Notebooks/Benchmark
  !pip install shap

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/Benchmark
     |████████████████████████████████| 356 kB 33.6 MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491650 sha256=2be20834dcd0083f84f2103c71a2bcffaf4ef91237c38cd0bcd4ece669606117
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built shap


In [2]:
import pandas as pd
df = pd.read_csv(r'results/Experiments_cnn_model.csv', index_col=['experiment'])
df['mean_loss'] = (df['mae_blind'] + df['rmse_blind'] + df['loss'] + df['mae'] + df['rmse'])/5
display(df.sort_values(['mean_loss']))

,layers,dropout_rate,activation,timesteps,batch_size,filters,kernel_size,pool_size,epochs,mae_blind,rmse_blind,loss,mae,rmse,loss.1,mae.1,rmse.1,train_time,mean_loss
experiment,,,,,,,,,,,,,,,,,,,
126,1,0.0,tanh,7,20,32,6,2,25,0.148089,0.173559,0.038269,0.025950,0.038863,[[0.22999752]],[[0.15595705]],[[0.23356634]],17.192366,0.084946
350,1,0.5,tanh,7,30,32,4,2,25,0.143444,0.169081,0.041282,0.029328,0.041735,[[0.24810451]],[[0.17626111]],[[0.25082539]],13.383295,0.084974
5,1,0.0,relu,7,10,32,6,1,25,0.151992,0.176716,0.037423,0.025195,0.038083,[[0.22491258]],[[0.15142328]],[[0.2288815]],24.245493,0.085882
2,1,0.0,relu,7,10,32,4,2,25,0.150791,0.175682,0.038572,0.026045,0.039223,[[0.23181624]],[[0.15653031]],[[0.23572828]],21.466384,0.086063
132,1,0.0,tanh,7,20,64,6,2,25,0.148496,0.174695,0.039964,0.028095,0.040506,[[0.24018251]],[[0.16884888]],[[0.24344092]],17.486611,0.086351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,1,0.0,relu,21,10,64,5,2,25,0.270759,0.291663,0.065785,0.052421,0.065471,[[0.3953705]],[[0.31504885]],[[0.39348042]],26.022124,0.149220
431,1,0.5,tanh,21,30,64,6,1,25,0.302388,0.325200,0.060003,0.047466,0.059605,[[0.3606196]],[[0.28526884]],[[0.35822642]],12.958520,0.158932
427,1,0.5,tanh,21,30,64,4,1,25,0.305317,0.328497,0.059799,0.047104,0.059376,[[0.35939103]],[[0.28309508]],[[0.35684871]],14.680101,0.160018


In [3]:
SEED = 91195003

import load_dataset as ld
import timeseries_preparation as tsp
import experiments as exp
import build_model as bm
import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit
import time
import numpy as np

'''
Just load the dataset
'''
df_ph = ld.load_ph_dataset(univariate=True, colab=COLAB)
display(df_ph.sort_values(['value']))

'''
Normalize it
'''
df_data = df_ph.copy()
scalers = tsp.data_normalization(df_data, norm_range=(0, 1))

'''
Set the TimeSeries parameters
'''
multisteps = 2
cv_splits = 3
#must respect the name of the arguments of the build_model function
#timesteps are mandatory
hyperparameters = {
    'layers': [1],
    'dropout_rate': [0.0],
    'activation': ['tanh'],
    'timesteps': [7],
    'batch_size': [20],
    'filters': [32],
    'kernel_size': [6],
    'pool_size': [2],
}
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=25, min_lr=0.00005),
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, mode='min', verbose=0, patience=40)]

param_list, param_names = exp.prepare_param_list(hyperparameters, bm.build_cnn_model)
model = exp.build_model(1, param_names, param_list[0], bm.build_cnn_model, SEED)
print(model.summary())

X, y = tsp.to_supervised(df_data, param_list[0][param_names.index('timesteps')])
tscv = TimeSeriesSplit(n_splits=cv_splits)
for train_index, test_index in tscv.split(X):
  train_idx, val_idx = tsp.split_dataset(train_index, perc=10)
  X_train, y_train = X[train_idx], y[train_idx] 
  X_val, y_val = X[val_idx], y[val_idx] 
  X_test, y_test = X[test_index], y[test_index]
  if callbacks:
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=25, batch_size=param_list[0][param_names.index('batch_size')], shuffle=False, verbose=1, callbacks=callbacks)
'''model_json = model.to_json()
print(model_json)
filename = 'models/lstm_' + time.strftime("%Y%m%d%H%M") + '.h5'
model.save_weights(filename)'''

,value
timestep,
2017-11-01,3.550
2016-05-20,4.040
2016-05-04,4.160
2017-11-15,4.165
2017-12-15,4.200
...,...
2019-11-25,8.800
2019-04-11,9.020
2019-11-24,9.040


Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 7, 1)]            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 2, 16)             112       
_________________________________________________________________
average_pooling1d (AveragePo (None, 1, 16)             0         
_________________________________________________________________
flatten (Flatten)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                544       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33

'model_json = model.to_json()\nprint(model_json)\nfilename = \'models/lstm_\' + time.strftime("%Y%m%d%H%M") + \'.h5\'\nmodel.save_weights(filename)'

In [5]:
def recursive_forecast(model, X_test, y_test, timesteps, multisteps, features, scaler, col_target_position, evaluation_timesteps=30):
    targer_scaler = scaler['value']    
    input_seq = X_test[-evaluation_timesteps:, :, :]
    labels_seq = y_test[-evaluation_timesteps:, :]
    results = list()
    for i in range(len(input_seq)-(multisteps-1)):  #we cannot go through the entire input because we are multistep forecasting!
        inp = input_seq[i].copy()
        print(inp)
        lab = labels_seq[i].copy()
        #for each step of the multistep
        labels = list()
        predictions = list()
        for step in range(1, multisteps+1):
            #reshape
            inp = inp.reshape(1, timesteps, features)
            lab = lab.reshape(1, 1)
            #predict the value for the next timestep
            yhat = model.predict(inp, verbose=0)
            #invert normalized values
            lab_inversed = targer_scaler.inverse_transform(lab)
            yhat_inversed = targer_scaler.inverse_transform(yhat)
            #print(lab_inversed, yhat_inversed)
            #store results
            labels.append(lab_inversed[0][0])
            predictions.append(yhat_inversed[0][0])
            #insert a new value into the input sequence to predict the next timestep.
            if step != multisteps:
                #add yhat to the input sequence
                new_line = input_seq[i+step][-1].copy()
                np.put(new_line, col_target_position, yhat)
                new_line = new_line.reshape(1, features)
                inp = np.concatenate((inp[0], new_line))
                inp = inp[-timesteps:]
                #update label to the next timestep
                lab = labels_seq[i+step]
        results.append((np.array(predictions)))
    return np.array(results)

#generated_data = np.random.uniform(low=6, high=9, size=9)
generated_data = [7.61, 7.075, 7.525, 6.9849, 7.105, 7.185, 7.04, 7.15, 7.165, 7.06]
generated_df = pd.DataFrame(generated_data, columns=['value'])
generated_df['value'] = scalers['value'].transform(generated_df[['value']])
X_generated, y_generated = tsp.to_supervised(generated_df , param_list[0][param_names.index('timesteps')])
generated_df['ph'] = scalers['value'].inverse_transform(generated_df[['value']])
display(generated_df)
print(recursive_forecast(model, X_generated, y_generated, param_list[0][param_names.index('timesteps')], 2, 1, scalers, generated_df.columns.get_loc('value')))

,value,ph
0,0.675541,7.6100
1,0.586522,7.0750
2,0.661398,7.5250
3,0.571531,6.9849
4,0.591514,7.1050
5,0.604825,7.1850
6,0.580699,7.0400
7,0.599002,7.1500
8,0.601498,7.1650
9,0.584027,7.0600


[[0.67554075]
 [0.58652246]
 [0.6613977 ]
 [0.57153076]
 [0.59151417]
 [0.6048253 ]
 [0.58069885]]
[[0.58652246]
 [0.6613977 ]
 [0.57153076]
 [0.59151417]
 [0.6048253 ]
 [0.58069885]
 [0.59900165]]
[[7.2576194 7.238274 ]
 [7.2209682 7.2423377]]
